# Variables

In [ ]:
ga_input = '/grid/home/nbourgeois/data/test_proteins/hidua/hidua.pep.fas' #sequence pep.fas Input
tissue = 'Brain_Cerebellum' # Tissue type for CoCoPuts
filename = 'test'

# Imports


In [2]:
import os
cwd = os.getcwd()

## Import the code for codon opt
pygad_loc = '/grid/home/nbourgeois/codonOpt'
os.chdir(pygad_loc)
import run_pygad
from run_pygad import *
from general_functions import *
os.chdir(cwd)


Using TensorFlow backend.


## Metrics

In [3]:
cai_on = True
bai_on = True
cpg_on = True
pas_on = False # PolyA
sps_on = False # Splicing


# Data

In [4]:
(keys, seqs) = readFasta(ga_input)
if len(seqs) == 1:
    aa_seq=str(seqs[0])

# Genetic Algorithm

In [5]:
tissue = 'Brain_Cerebellum' # Tissue type for CoCoPuts
filename = 'test'

In [6]:
%%time 

prefix=filename
aa_seq=aa_seq
tissue=tissue
ga = run_GA(aa_seq, 
        tissue, 
        generations=1000,
        cai_on = True, 
        bai_on = True, 
        cpg_on = True,
        threads=10)


CPU times: user 4min 58s, sys: 4.09 s, total: 5min 2s
Wall time: 5min 23s


In [7]:
# Returning the details of the best solution.
solution, solution_fitness, solution_idx = ga.best_solution()
seq_aa = ''.join([ga.codon_to_int[x] for x in solution])
print("NT Sequence of the best solution : {seq_aa}".format(seq_aa=seq_aa))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

if ga.best_solution_generation != -1:
    print("Best fitness value reached after {best_solution_generation} generations.".format(best_solution_generation=ga.best_solution_generation))

NT Sequence of the best solution : ATGAGGCCTCTGAGGCCCAGAGCCGCCCTGCTGGCCCTGCTGGCCAGCCTGCTGGCAGCTCCTCCCGTGGCCCCCGCAGAGGCTCCCCACCTGGTGCACGTGGATGCCGCTCGAGCCCTTTGGCCTCTGAGACGCTTCTGGCGGTCAACCGGCTTCTGTCCCCCTCTTCCACACAGCCAAGCAGATCAGTACGTGCTGAGCTGGGATCAGCAGCTGAACCTGGCCTATGTAGGGGCCGTCCCACATAGGGGGATCAAGCAGGTGCGGACCCACTGGCTGCTGGAGCTGGTGACTACCCGTGGCTCAACAGGGCGGGGGCTGTCCTACAATTTTACCCACTTGGACGGATACCTGGACCTGCTGCGGGAGAACCAGCTGCTGCCTGGCTTTGAGCTGATGGGATCTGCGTCTGGCCACTTCACAGACTTCGAGGACAAGCAGCAGGTGTTCGAATGGAAAGATCTTGTCAGCAGCCTGGCCAGGAGGTACATCGGCCGGTACGGTCTGGCCCATGTGAGCAAATGGAACTTTGAGACCTGGAATGAGCCTGACCACCACGACTTCGACAATGTGTCCATGACGATGCAGGGTTTCCTGAATTACTACGATGCATGCTCTGAGGGCCTCAGAGCCGCCTCCCCAGCCCTGAGGCTGGGCGGCCCGGGAGACTCCTTCCACACCCCACCCCGCAGCCCCCTGAGTTGGGGCCTGCTGCGGCACTGCCACGACGGGACCAATTTCTTCACCGGCGAGGCTGGGGTGAGGCTGGACTACATCTCCCTGCACAGGAAGGGGGCCCGCAGCAGCATCAGCATCCTGGAGCAGGAGAAAGTGGTGGCCCAGCAGATCAGGCAGCTGTTCCCTAAGTTCGCTGACACCCCCATCTACAATGATGAGGCTGATCCGCTGGTGGGGTGGAGCCTGCCCCAGCCCTGGCGGGCTGACGTGACTTACGCCGCTATGGT

In [ ]:
ga.plot_new_solution_rate()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_106317/3685778001.py", line 1, in <module>
    ga.plot_new_solution_rate()
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/pygad/pygad.py", line 3196, in plot_new_solution_rate
    raise RuntimeError("The plot_new_solution_rate() method works only when save_solutions=True in the constructor of the pygad.GA class.")
RuntimeError: The plot_new_solution_rate() method works only when save_solutions=True in the constructor of the pygad.GA class.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
Attrib

In [ ]:
ga.plot_genes()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_106317/1988088177.py", line 1, in <module>
    ga.plot_genes()
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/pygad/pygad.py", line 3275, in plot_genes
    raise RuntimeError("The plot_genes() method with solutions='all' can only be called if 'save_solutions=True' in the pygad.GA class constructor.")
RuntimeError: The plot_genes() method with solutions='all' can only be called if 'save_solutions=True' in the pygad.GA class constructor.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/software/anaconda3/envs/bfx-gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeE